In [25]:
# This piece of code enables display of multiple output from one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Data Description

The data has been split into two groups:

training set (train.csv)
test set (test.csv)
The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

Data Dictionary
Variable	Definition	Key
survival	Survival	0 = No, 1 = Yes
pclass	Ticket class	1 = 1st, 2 = 2nd, 3 = 3rd
sex	Sex	
Age	Age in years	
sibsp	# of siblings / spouses aboard the Titanic	
parch	# of parents / children aboard the Titanic	
ticket	Ticket number	
fare	Passenger fare	
cabin	Cabin number	
embarked	Port of Embarkation	C = Cherbourg, Q = Queenstown, S = Southampton
Variable Notes
pclass: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

age: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

sibsp: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

parch: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.


Reference: [Kaggle](https://www.kaggle.com/c/titanic/data?select=train.csv)

In [26]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn import metrics

In [28]:
df = pd.read_csv("titanic_train.csv") 
df_test = pd.read_csv("titanic_test.csv")
df.shape
df_test.shape

(891, 12)

(418, 11)

### Data Quality

In [29]:
df = df.drop(['Ticket','Cabin'], axis=1)
# Impute median Age for NA Age values
imputed_age = df["Age"].median()
new_age_var = np.where(df["Age"].isnull(), imputed_age, df["Age"])

df["Age"] = new_age_var
df = df.dropna()
df.shape

(889, 10)

In [30]:
df_test.isnull().sum()
df_test = df_test.dropna()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [31]:
categorical = ['Pclass', 'Sex', 'Embarked']
numerical = ['Age', 'Fare', 'SibSp', 'Parch']

### Model Fitting and Prediction

In [37]:
def train(df, y, C=1.0):
    cat = df[categorical + numerical].to_dict(orient='records')
    
    dv = DictVectorizer(sparse=False)
    dv.fit(cat)

    X = dv.transform(cat)

    model = LogisticRegression(solver='liblinear', C=C)
    model.fit(X, y)

    return dv, model
def predict(xmatrix, dv, model):
    cat = xmatrix[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(cat)

    y_pred = model.predict_proba(X)[:, 1]

    return y_pred

def predict_single(passenger, dv, model):
    cat = passenger[categorical + numerical].to_dict(orient='records')
    
    X = dv.transform(passenger)

    y_pred = model.predict_proba(X)[:, 1]

    return y_pred[0]



In [38]:
y_train = df.Survived.values

X_train=df.copy()
del X_train['Survived']


In [39]:
dv, model = train(df, y_train, C=0.1)

y_pred = predict(df_test, dv, model)

In [40]:
passenger = {
        'PassengerId':'902',
        'Pclass': '3',
        'Name':'John Doe',
         'Sex':'male',
         'Age': 28.0,
        'SibSp': 0.0,
        'Parch':0.0,
        'Fare': 8.0,
        'Embarked':'S'
    
}

In [41]:
predict_single(passenger, dv, model)

TypeError: unhashable type: 'list'

In [42]:
invd=pd.DataFrame([passenger])
invdp=predict(invd, dv, model)
invdp[0]

0.432649804682505

### Saving the model

In [47]:
import pickle

with open('titanic-model.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)
    f_out.close()